## We give access the program to our Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## First we install bigdl and spark, and exit to reload and have the package well installed


In [ ]:
!pip -q install bigdl-spark3
exit()

## We initialize a spark session

In [ ]:
from pyspark.sql import SparkSession
from bigdl.dllib.nncontext import init_nncontext

sc = init_nncontext(cluster_mode="local")
spark = SparkSession.builder.getOrCreate()

Current pyspark location is : /usr/local/lib/python3.8/dist-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /usr/local/lib/python3.8/dist-packages/bigdl/share/core/lib/all-2.2.0.jar:/usr/local/lib/python3.8/dist-packages/bigdl/share/dllib/lib/bigdl-dllib-spark_3.1.3-2.2.0-jar-with-dependencies.jar:/usr/local/lib/python3.8/dist-packages/bigdl/share/friesian/lib/bigdl-friesian-spark_3.1.3-2.2.0-jar-with-dependencies.jar:/usr/local/lib/python3.8/dist-packages/bigdl/share/orca/lib/bigdl-orca-spark_3.1.3-2.2.0-jar-with-dependencies.jar pyspark-shell 
Successfully got a SparkContext


## We download the train and test images in 2 dataframes, and resize in the loading using resizH and resizeW. We alsa added a label column too where 1 means Normal and 2 Pneumonia

In [ ]:
from bigdl.dllib.nnframes import NNImageReader
from pyspark.sql.functions import lit

datadir = "/content/drive/MyDrive/Colab Notebooks/chest_xray"

train = \
  NNImageReader.readImages(f"{datadir}/train/NORMAL/", sc, image_codec=0, resizeH=250, resizeW=250)\
  .withColumn("label", lit(1.0))\
  .union(
      NNImageReader.readImages(f"{datadir}/train/PNEUMONIA/", sc, image_codec=0, resizeH=250, resizeW=250)\
      .withColumn("label", lit(2.0)))

test = \
  NNImageReader.readImages(f"{datadir}/test/NORMAL/", sc, image_codec=0, resizeH=250, resizeW=250)\
  .withColumn("label", lit(1.0))\
  .union(
      NNImageReader.readImages(f"{datadir}/test/PNEUMONIA/", sc, image_codec=0, resizeH=250, resizeW=250)\
      .withColumn("label", lit(2.0)))

val = \
  NNImageReader.readImages(f"{datadir}/val/NORMAL/", sc, image_codec=0, resizeH=250, resizeW=250)\
  .withColumn("label", lit(1.0))\
  .union(
      NNImageReader.readImages(f"{datadir}/val/PNEUMONIA/", sc, image_codec=0, resizeH=250, resizeW=250)\
      .withColumn("label", lit(2.0)))

## We can see how all the images now are 250 x 250 in train and test

In [ ]:
train.select(["image.height", "image.width"]).show(15)

+------+-----+
|height|width|
+------+-----+
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
+------+-----+
only showing top 15 rows



In [ ]:
test.select(["image.height", "image.width"]).show(15)

+------+-----+
|height|width|
+------+-----+
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
|   250|  250|
+------+-----+
only showing top 15 rows



## We have 2 columns, 1 for the labels and the other for the images

In [ ]:
train.dtypes, test.dtypes

([('image',
   'struct<origin:string,height:int,width:int,nChannels:int,mode:int,data:binary>'),
  ('label', 'double')],
 [('image',
   'struct<origin:string,height:int,width:int,nChannels:int,mode:int,data:binary>'),
  ('label', 'double')])

## Now we are going to convert the Image column in a feature vector

In [ ]:
import pyspark.sql.functions as F
from pyspark.ml.image import ImageSchema
from pyspark.ml.linalg import DenseVector, VectorUDT

ImageSchema.imageFields

img2vec = F.udf(lambda x: DenseVector(ImageSchema.toNDArray(x).flatten()), VectorUDT())



train_final=train.withColumn('features', img2vec("image"))
test_final=test.withColumn('features', img2vec("image"))


## We only want to have at the end the features and the label columns

In [ ]:
train_final= train_final.select(["features", "label"])
test_final= test_final.select(["features", "label"])


## After have the final data frames we are going to use a Multilayer Perceptron classifier to make the model

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.feature import StandardScaler
mlpc=MultilayerPerceptronClassifier( featuresCol='features',labelCol='label',layers = [4,16,2],\
                                    maxIter=1000,blockSize=8,seed=7,solver='gd')

## At the end we obtained an error on the fitting step like in the Guide that we had, we tryed to find everywhere a solution, and we asked for help, but at the end we couldn't find a solution for the error. So we couldn't obtain any result

In [ ]:
ann = mlpc.fit(train_final)

Py4JJavaError: ignored

## If the fitting step were succesfull we had done the next steps to see how it works the model and to calculate the F1 value and the accuracy and this would have been the result. We also would have tryed to fit with other models like Logistic regression and random forest, for this last one, we had create a dataframe with all training, validation and test and make the random forest with this bigger dataset

In [ ]:
pred = ann.transform(test_final)
evaluator = MulticlassClassificationEvaluator(labelCol='Class ',predictionCol='prediction',metricName='f1')
ann_f1 = evaluator.evaluate(pred)
ann_f1

In [ ]:

evaluator = MulticlassClassificationEvaluator(labelCol='Class ',predictionCol='prediction',metricName='accuracy')
accuracy = evaluator.evaluate(pred)
accrucy